# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [93]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [95]:
# 1) Seu código aqui
# 1) Repita os mesmos passos feitos em aula, 
#    mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.

url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Munic%C3%ADpio'
pvds_rj = requests.get(url).json()
df_pvds_rj = pd.DataFrame(data['items'])
df_pvds_rj.head()

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,65267,Município,Miguel Pereira,3302908,RJ,PVL02.002355/2023-18,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,10000000.00,0,1,13/07/2023
1,58189,Município,Nova Friburgo,3303401,RJ,PVL02.003661/2022-91,Assinado pelo interessado (retificação),17944.101850/2022-91,2024-07-24T20:29:41Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,30000000.00,0,0,12/08/2024
2,11005,Município,Araruama,3300209,RJ,None,Arquivado por decurso de prazo,17944.001334/2011-12,2012-12-21T02:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,33593206.91,0,0,09/01/2013
3,13780,Município,Nova Friburgo,3303401,RJ,None,Deferido,19405.000082/2002-38,2002-11-25T02:00:00Z,Operação contratual interna,PMAT,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,2286000.00,1,0,18/12/2002
4,5618,Município,Nova Iguaçu,3303500,RJ,None,Arquivado,17944.000156/2010-13,2010-12-03T02:00:00Z,Operação contratual interna,PAC - Saneamento - 2009 A,Instituição Financeira Nacional,Caixa Econômica Federal,Real,60737300.00,0,0,15/12/2010


In [97]:
df_pvls_rj['uf'].value_counts()

uf
RJ    268
Name: count, dtype: int64

In [99]:
# 2) Seu código aqui
status_counts = df_pvls_rj['status'].value_counts()
top_status = status_counts.head(3)
print(f'Conforme a base de dados, as três maiores frequencias são os {top_status}')
      

Conforme a base de dados, as três maiores frequencias são os status
Deferido        106
Arquivado        53
Regularizado     26
Name: count, dtype: int64


In [103]:
# 3) Seu código aqui
df_pvls_rj['ano'] = df_pvls_rj['data_status'].str.slice(-4,None)

In [105]:
df_pvls_rj['ano'].value_counts().sort_index(ascending=True)

ano
2002     4
2003     1
2004     1
2006     4
2007    29
2008    37
2009     9
2010    19
2011    10
2012    13
2013    15
2014    20
2015     8
2016     4
2017     5
2018     4
2019    16
2020    15
2021     5
2022    11
2023    28
2024    10
Name: count, dtype: int64

####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [113]:
#1) Seu código aqui
def consultar_PVL(uf:str='SP', tipo_interessado:str='Estado') -> pd.DataFrame:
    url = 'https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={}&tipo_interessado={}'.format(
        uf, 
        tipo_interessado)
    r_uf = requests.get(url)  
    return pd.DataFrame(r_uf.json()['items'])

consultar_PVL('RJ', 'Município')

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,65267,Município,Miguel Pereira,3302908,RJ,PVL02.002355/2023-18,Deferido (PVL-IF),None,None,Operação contratual interna,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.000000e+07,0,1,13/07/2023
1,58189,Município,Nova Friburgo,3303401,RJ,PVL02.003661/2022-91,Assinado pelo interessado (retificação),17944.101850/2022-91,2024-07-24T20:29:41Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Caixa Econômica Federal,Real,3.000000e+07,0,0,12/08/2024
2,11005,Município,Araruama,3300209,RJ,None,Arquivado por decurso de prazo,17944.001334/2011-12,2012-12-21T02:00:00Z,Operação contratual interna,PAC 2 - Pró-Transporte - Pavimentação e Qualif...,Instituição Financeira Nacional,Caixa Econômica Federal,Real,3.359321e+07,0,0,09/01/2013
3,13780,Município,Nova Friburgo,3303401,RJ,None,Deferido,19405.000082/2002-38,2002-11-25T02:00:00Z,Operação contratual interna,PMAT,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,2.286000e+06,1,0,18/12/2002
4,5618,Município,Nova Iguaçu,3303500,RJ,None,Arquivado,17944.000156/2010-13,2010-12-03T02:00:00Z,Operação contratual interna,PAC - Saneamento - 2009 A,Instituição Financeira Nacional,Caixa Econômica Federal,Real,6.073730e+07,0,0,15/12/2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,62211,Município,Valença,3306107,RJ,PVL02.008444/2022-97,Deferido (PVL-IF),None,None,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco do Brasil S/A,Real,7.148000e+06,0,1,29/06/2023
264,8684,Município,Rio de Janeiro,3304557,RJ,None,Deferido,17944.000824/2011-93,2011-11-16T02:00:00Z,Operação contratual interna,Saneamento básico,Instituição Financeira Nacional,Caixa Econômica Federal,Real,1.432000e+08,1,0,03/01/2012
265,11315,Município,Maricá,3302700,RJ,None,Deferido,17944.001410/2010-09,2011-01-10T02:00:00Z,Operação contratual interna,PMAT,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,1.051389e+07,1,0,19/01/2011
266,10226,Município,Conceição de Macabu,3301405,RJ,None,Arquivado,17944.001149/2008-14,2008-08-14T03:00:00Z,Operação contratual interna,Caminho da Escola 2,Instituição Financeira Nacional,Banco do Brasil S/A,Real,4.007000e+05,0,0,30/10/2008


In [123]:
# 2) Seu código aqui
df_estado_mg = consultar_PVL('MG', 'Estado')
print("Quantidade de solicitações para Minas Gerais com status em 'Arquivado por decurso de prazo':")
print(df_estado_mg[df_estado_mg['status'] == 'Arquivado por decurso de prazo'].shape[0])

Quantidade de solicitações para Minas Gerais com status em 'Arquivado por decurso de prazo':
1


In [125]:
# 3) Seu código aqui
df_estado_ba = consultar_PVL('BA', 'Município')
municipio_mais_solicitacoes = df_estado_ba['interessado'].value_counts().idxmax()
numero_solicitacoes = df_estado_ba['interessado'].value_counts().iloc[0]
print(f'O município da Bahia com mais solicitações deferidas: {municipio_mais_solicitacoes.upper()} com {numero_solicitacoes} solicitações deferidas.')


O município da Bahia com mais solicitações deferidas: CAMAÇARI com 21 solicitações deferidas.


In [119]:
# 4) Seu código aqui
df_pvls_estado_ba = consultar_PVL('BA', 'Estado')
df_pvls_estado_ba.to_csv('pvls_estado_ba.csv', index=False)